In [177]:
import pandas as pd
import numpy as np
from datetime import datetime

# Storm Events (SWDI)

You can find the data files from https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/ that has the severe weather records from 1950 - 2023 for the United States.

The detailed description of the format can be found here https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/Storm-Data-Bulk-csv-Format.pdf

In [178]:
df = pd.read_csv("StormEvents_details-ftp_v1.0_d2023_c20240317.csv")

In [179]:
df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,202310,25,230,202310,27,551,186682,1145781,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Public reports 7.5 inches at Black Tiger Bay C...,CSV
1,202310,25,230,202310,27,1437,186682,1145783,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Local Police Department relays storm total sno...,CSV
2,202310,25,230,202310,27,1126,186682,1145784,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Public reports 10 inches of storm total snowfa...,CSV
3,202310,25,230,202310,27,1301,186682,1145796,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Emergency Manager reports 6 inches of storm to...,CSV
4,202310,25,230,202310,27,600,186682,1145884,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",CoCoRaHS Station ND-GF-23 reports 8.8 inches o...,CSV


In [180]:
df.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [181]:
select_cols = ['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'STATE_FIPS','EVENT_TYPE', 'CZ_FIPS']
select_df = df[select_cols]
select_df.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'STATE_FIPS', 'EVENT_TYPE', 'CZ_FIPS'],
      dtype='object')

In [182]:
select_df.info(show_counts=True, memory_usage=True, verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74909 entries, 0 to 74908
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BEGIN_YEARMONTH  74909 non-null  int64 
 1   BEGIN_DAY        74909 non-null  int64 
 2   BEGIN_TIME       74909 non-null  int64 
 3   END_YEARMONTH    74909 non-null  int64 
 4   END_DAY          74909 non-null  int64 
 5   END_TIME         74909 non-null  int64 
 6   STATE_FIPS       74909 non-null  int64 
 7   EVENT_TYPE       74909 non-null  object
 8   CZ_FIPS          74909 non-null  int64 
dtypes: int64(8), object(1)
memory usage: 5.1+ MB


In [183]:
select_df.shape

(74909, 9)

In [184]:
select_df.isnull().sum()

BEGIN_YEARMONTH    0
BEGIN_DAY          0
BEGIN_TIME         0
END_YEARMONTH      0
END_DAY            0
END_TIME           0
STATE_FIPS         0
EVENT_TYPE         0
CZ_FIPS            0
dtype: int64

# Tables of storm events

In our tables, there are 5 columns, fips_code(refer to Outage Events for the details), event types(various severe weather types), starting time of the events, end time of the events, and their duration.

Question: Do we need the detailed location (begin/end latitude/longitude) of each event? 
    They provide more geometrical/topological imformation. But we don't have the specific trajectories. 

Question: These files have much more information about tornado events than other events. 
    Should we generate a table specifically for tornado events and have a focus on them?



In [185]:
def combine_fips(state_fip, cz_fip):
    return "0"*(2-len(state_fip)) + state_fip + "0"*(3-len(cz_fip)) + cz_fip

def combine_time(yearmonth, day, time):
    hour = 0
    if len(time) >= 3:
        hour = int(time[:-2])
    minute = int(time[-2:])
    return str(datetime(int(yearmonth[:4]), int(yearmonth[4:]), int(day), hour, minute))



rows_lst = []
for inx in select_df.index:
    new_dict = {}
    new_dict.update({"fips_code": combine_fips( select_df["STATE_FIPS"][inx].astype(str), select_df["CZ_FIPS"][inx].astype(str) ) })
    new_dict.update({"event_type": select_df["EVENT_TYPE"][inx]})
    new_dict.update({"start": combine_time(select_df["BEGIN_YEARMONTH"][inx].astype(str), select_df["BEGIN_DAY"][inx].astype(str), select_df["BEGIN_TIME"][inx].astype(str) )   })
    new_dict.update({"end": combine_time(select_df["END_YEARMONTH"][inx].astype(str), select_df["END_DAY"][inx].astype(str), select_df["END_TIME"][inx].astype(str) )   })
    diff_time = datetime.fromisoformat(new_dict["end"]) - datetime.fromisoformat(new_dict["start"])
    diff_minutes = int(diff_time.total_seconds() / 60)
    new_dict.update({"duration": diff_minutes})
    rows_lst.append(new_dict)

storm_events = pd.DataFrame(rows_lst)

storm_events.head()
    

,fips_code,event_type,start,end,duration
0,38014,Heavy Snow,2023-10-25 02:30:00,2023-10-27 05:51:00,3081
1,38024,Heavy Snow,2023-10-25 02:30:00,2023-10-27 14:37:00,3607
2,38026,Heavy Snow,2023-10-25 02:30:00,2023-10-27 11:26:00,3416
3,38028,Heavy Snow,2023-10-25 02:30:00,2023-10-27 13:01:00,3511
4,38027,Heavy Snow,2023-10-25 02:30:00,2023-10-27 06:00:00,3090


In [186]:
storm_events[storm_events.fips_code == "12073"]

,fips_code,event_type,start,end,duration
672,12073,Heat,2023-07-06 14:00:00,2023-07-06 14:00:00,0
7541,12073,Tornado,2023-01-25 10:23:00,2023-01-25 10:27:00,4
7544,12073,Thunderstorm Wind,2023-01-25 10:36:00,2023-01-25 10:36:00,0
16965,12073,Thunderstorm Wind,2023-04-15 19:37:00,2023-04-15 19:37:00,0
17227,12073,Thunderstorm Wind,2023-04-15 19:45:00,2023-04-15 19:45:00,0
...,...,...,...,...,...
66078,12073,Thunderstorm Wind,2023-09-07 18:35:00,2023-09-07 18:35:00,0
66332,12073,Thunderstorm Wind,2023-09-07 18:45:00,2023-09-07 18:45:00,0
66333,12073,Thunderstorm Wind,2023-09-07 18:40:00,2023-09-07 18:40:00,0
66334,12073,Thunderstorm Wind,2023-09-07 18:45:00,2023-09-07 18:45:00,0



Quesiton: Many events have duration 0 (or perhaps less than one minute?).
    What does this mean? Should we remove these events?
    

In [187]:
storm_events = storm_events[storm_events['duration'] != 0]
storm_events[storm_events.fips_code == "12073"]

,fips_code,event_type,start,end,duration
7541,12073,Tornado,2023-01-25 10:23:00,2023-01-25 10:27:00,4
18557,12073,Thunderstorm Wind,2023-04-27 17:24:00,2023-04-27 17:36:00,12
19674,12073,Drought,2023-04-01 00:00:00,2023-04-18 07:00:00,24900
20053,12073,Thunderstorm Wind,2023-04-27 17:18:00,2023-04-27 17:40:00,22
24153,12073,Thunderstorm Wind,2023-03-10 14:35:00,2023-03-10 14:40:00,5
27778,12073,Drought,2023-03-28 07:00:00,2023-03-31 23:59:00,5339
53911,12073,Tropical Depression,2023-08-29 13:00:00,2023-08-30 15:00:00,1560
66589,12073,Flash Flood,2023-12-02 10:49:00,2023-12-02 13:00:00,131


In [188]:
storm_events.to_csv("storm_events_2023.csv", index=False)